In [1]:
# pip install selenium
# pip install webdriver_manager
# pip install nltk
# pip install gender_guesser

In [2]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import pandas as pd
import time
import gender_guesser.detector as gender

In [3]:
driver = webdriver.Chrome(ChromeDriverManager().install())

[WDM] - Current google-chrome version is 87.0.4280
[WDM] - Get LATEST driver version for 87.0.4280


[WDM] - Driver [C:\Users\Noam\.wdm\drivers\chromedriver\win32\87.0.4280.88\chromedriver.exe] found in cache


In [4]:
########################################################

In [5]:
csv_list = []

In [6]:
def Chall_Names_List(driver, soup):
    chall_names_list = []

    titles = soup.find_all("div",{"class":"challenge-title"})
    for title in titles:
        t=title.find('a')['href']
        chall_names_list.append(t[13:])
        
    
    return chall_names_list

In [7]:
def make_challenges_hrefs_list():
    

    challenges_href = []
    driver.get('https://coderbyte.com/challenges')
    driver.find_element_by_css_selector("input[name='free']").click()
    
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    
    challenges_names = Chall_Names_List(driver, soup)
    
    
    challs_cards = soup.find_all("div",{"class":"challenge-actions"})
   
    for card in challs_cards:
      
        sol_href = card.find_all('a')
      
        string_list = str(sol_href).split()
        space = False
        for string in string_list:
            if(space == True):
                newString = newString +"%20"+ string
                if(string[-4:]=="View"):
                    space = False
                    challenges_href.append(newString[:-6])
            if(string.find('/solution')!=-1):
                space = True
                newString = "https://coderbyte.com/" + string[6:]
                

    return challenges_href, challenges_names

In [8]:
def make_langs_list():
    langs_list = ['Java','JavaScript','C','Python','PHP','Python3','Go',"Ruby","Csharp","Swift"]
    
    return langs_list

In [9]:
Challenges_list = make_challenges_hrefs_list()[0]
Challenges_Names = make_challenges_hrefs_list()[1]
Languages_list = make_langs_list()

# ### Challenges check: ###
# print(len(Challenges_list))
# for href in Challenges_list:
#     print(href)

# ### Challenges names check: ###
# print(len(Challenges_Names))
# for name in Challenges_Names:
#     print(name)


# ### Languages check: ###
# print(len(Languages_list))
# for lang in Languages_list:
#     print(lang)


In [10]:
d = gender.Detector()
def find_gender(name):
    male_prob = 0
    female_prob = 0
    Gender = None
    CAP_name = name[0]
    name = name[1:]
    CAP_name=CAP_name.capitalize()
    name = CAP_name+name
    for i in range(len(name)):
        if (i<3): continue
        name = name[:-1]
        if(d.get_gender(name)=="male"): male_prob = male_prob +1
        if(d.get_gender(name)=="female"): female_prob = female_prob +1
    if(male_prob>female_prob): Gender = "male"
    elif(male_prob<female_prob): Gender = "female"
    return Gender

In [11]:
########################################################

In [12]:

# def login():   
#     driver = webdriver.Chrome(ChromeDriverManager().install())
#     url = 'https://www.coderbyte.com/solution/Bracket%20Matcher'
#     userName = 'nnnnb'
#     password = '12334'
#     driver.get(url)
#     time.sleep(5)
#     driver.find_element_by_class_name("login-link").click()
#     time.sleep(2)
#     driver.find_element_by_class_name("mode-toggle-option").click()
#     time.sleep(1)
#     driver.find_element_by_class_name("login-field-input").send_keys(userName)
#     time.sleep(1)
#     driver.find_element_by_css_selector("input[type='password']").send_keys(password)
#     time.sleep(1)
#     xpath = '//*[@id="login"]/div/div/div/div[1]/div[3]/button'
#     driver.find_element_by_xpath(xpath).click()
#     time.sleep(2)




    
    
# def current_Challange_Name(soup):
#     url = 'https://www.coderbyte.com/'
#     chall = soup.find("div",{"class":"leaderboard-controls"})
#     chall_href = chall.find('a')['href']
#     url+=chall_href
#  #   print(url)
#     chall_driver = webdriver.Chrome(ChromeDriverManager().install())
#     chall_driver.get(url)
#     chal_html = chall_driver.page_source
#     chal_soup = BeautifulSoup(chal_html, 'html.parser')
#     chall_div = chal_soup.find("div",{"class":"challenges-wrap"})
#     chall_span = chall_div.find('p').text
#     chall_driver.quit() 
#     return chall_span[6:]
       
    

In [13]:
#login()

In [14]:
def selectLang(lang,url):
    
    driver.get(url)
    time.sleep(3)
        
    xpath = '//*[@id="challenge_leaderboard"]/div/div/div[2]/div/div[2]/span'
    driver.find_element_by_xpath(xpath).click()
    time.sleep(1)
    driver.find_element_by_css_selector("input[type='text']").send_keys(lang)
    time.sleep(1)
    driver.find_element_by_css_selector("input[type='text']").send_keys(Keys.ENTER)



    
def data_process(html, lang,i):
    clean_data_list = []
    soup = BeautifulSoup(html, 'html.parser')
    cards = soup.find_all("div", {"class": "comment card"})

    for card in cards:
        data_dict = {}
        sol_code = card.find("code",{"class":"hljs"})
        sol_card = card.find("div",{"class":"submitter"})
        User_name = sol_card.find('a').text
        
        data_dict['User_Name'] = User_name
        data_dict['User_img'] = sol_card.find('img')['src']
        data_dict['User_url'] = sol_card.find('a')['href']
        data_dict['Challange_Name'] = Challenges_Names[i]
        data_dict['Programming_Lang'] = lang
        data_dict['Sol_Code'] = sol_code.get_text()
        data_dict['User_Score'] = sol_card.find('strong').text.strip()
        data_dict["User's_Comments"] = card.find("div",{"class":"comment-score-number"}).text
        
        data_dict["User_Gender"] = find_gender(User_name)

        
        clean_data_list.append(data_dict)
    
    return clean_data_list

In [15]:
def make_df(lang_name,url,i):
    selectLang(lang_name,url)
    html = driver.page_source
    clean_data_list = data_process(html,lang_name,i)
    df = pd.DataFrame(clean_data_list)
    
    return df

In [16]:
def df_to_csv(csv_file_name,df):
    csv_file_name = csv_file_name  + ".csv"
    df.to_csv(csv_file_name)
    csv_list.append(csv_file_name)

In [17]:
csv_file_name = ""
data_df  = pd.DataFrame()
for c in range(len(Challenges_list)):
    url = Challenges_list[c]
    csv_file_name = str(c)
    for l in range(len(Languages_list)):
        lang_name = Languages_list[l]
        csv_file_name = csv_file_name + str(l)
        df = make_df(lang_name,url,c)
        #df_to_csv(csv_file_name,df)
        data_df = pd.concat([data_df,df], ignore_index=True)


In [18]:
driver.quit()

In [19]:
data_df

,User_Name,User_img,User_url,Challange_Name,Programming_Lang,Sol_Code,User_Score,User's_Comments,User_Gender
0,TomGo,https://s3-us-west-1.amazonaws.com/coderbytepr...,/profile/TomGo,Min Window Substring,Java,import java.util.*; \nimport java.io.*;\n\ncla...,10,4,male
1,jgrier,https://s3-us-west-1.amazonaws.com/coderbytepr...,/profile/jgrier,Min Window Substring,Java,import java.util.*; \nimport java.io.*;\n\ncla...,10,0,None
2,buddhaben,https://s3-us-west-1.amazonaws.com/coderbytepr...,/profile/buddhaben,Min Window Substring,Java,import java.util.*; \nimport java.io.*;\n\ncla...,9,0,male
3,dumel20,https://s3-us-west-1.amazonaws.com/coderbytepr...,/profile/dumel20,Min Window Substring,Java,import java.util.*; \nimport java.io.*;\n\ncla...,10,0,None
4,sameerkhan2k1,https://s3-us-west-1.amazonaws.com/coderbytepr...,/profile/sameerkhan2k1,Min Window Substring,Java,import java.util.*; \nimport java.io.*;\n\ncla...,9,0,male
...,...,...,...,...,...,...,...,...,...
2868,mpalm1234,https://s3-us-west-1.amazonaws.com/coderbytepr...,/profile/mpalm1234,SQL Member Count,Swift,/* write your SQL query below */\n\nSELECT Rep...,10,0,None
2869,mag31903034,https://s3-us-west-1.amazonaws.com/coderbytepr...,/profile/mag31903034,SQL Member Count,Swift,/* write your SQL query below */\nSELECT a.rep...,10,-1,None
2870,Hucast734,https://s3-us-west-1.amazonaws.com/coderbytepr...,/profile/Hucast734,SQL Member Count,Swift,/* write your SQL query below */\nSELECT * FRO...,10,-1,None
2871,Balaji468,https://s3-us-west-1.amazonaws.com/coderbytepr...,/profile/Balaji468,SQL Member Count,Swift,/* write your SQL query below */\n\n\nSELECT R...,10,-1,None


In [22]:
pwd

'C:\\Users\\Noam'

In [23]:
df_to_csv("project_data",data_df)

In [24]:
data_df.User_Gender.isnull().sum()

1827

In [25]:
data_df.User_Gender.describe()

count     1046
unique       2
top       male
freq       795
Name: User_Gender, dtype: object